In [1]:
import numpy as np
import pandas as pd

In [2]:
organs = np.array(["bra", "hea", "int", "liv", "lun", "mus", "ski"])

fpkm_data = pd.read_csv("data/fpkm_all_anno.xls", sep='\t', header=0)
gene_lists = {}
for organ in organs:
    file_name = "data/deg_id/{}_union_deg_list.csv".format(organ)
    gene_lists[organ] = pd.read_csv(file_name, sep=',', header=0)
fpkm_data = fpkm_data.set_index("name")

/var/folders/pt/9f05dd_s4_lfhz62v29wfmtc0000gn/T/ipykernel_76756/1636768533.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168) have mixed types. Specify dtype option on import or set low_memory=False.
  fpkm_data = pd.read_csv("data/fpkm_all_anno.xls", sep='\t', header=0)


In [46]:
import itertools

def extract_df(fpkm_data, x, gene_set, filter=False):
    sample_name_list = []
    for sample_name in fpkm_data.columns:
        if (x in sample_name):
            sample_name_list.append(sample_name)
    if filter:
        # print(fpkm_data[sample_name_list].loc[gene_set])
        # print(np.min(fpkm_data[sample_name_list].loc[gene_set], axis=1) > 1)
        gene_set = gene_set[np.min(fpkm_data[sample_name_list].loc[gene_set], axis=1) > 1]
        # gene_set = gene_set[]

    sample_name_list.append("GeneName")
    return fpkm_data[sample_name_list].loc[gene_set].set_index('GeneName'), gene_set

for x, y in itertools.combinations(organs, 2):
    # print(gene_lists[x]["{}_degs_uni".format(x)])
    gene_set = np.union1d(np.array(gene_lists[x]["{}_degs_uni".format(x)]), np.array(gene_lists[y]["{}_degs_uni".format(y)]))
    # build x
    # print(extract_df(fpkm_data, x, gene_set))
    df_x, gene_set = extract_df(fpkm_data, x, gene_set, filter=True) 
    df_y, gene_set = extract_df(fpkm_data, y, gene_set)
    df_x.T.to_csv("data/input/{}_vs_{}-{}.csv".format(x, y, x), index_label=False, sep="\t")
    df_y.T.to_csv("data/input/{}_vs_{}-{}.csv".format(x, y, y), index_label=False, sep="\t")
    # print(gene_set[:10])
    # break
    # break

processing input with R scripts

In [66]:
for x, y in itertools.combinations(organs, 2):
    candidate_gene = pd.read_csv("result/{} X {} ranked by sig score".format(x, y), sep="\t").set_index("Gene_symbol")
    # candidatcandidate_gene
    # print(candidate_gene)
    # print(fpkm_data)
    col_list = []
    for sample_name in fpkm_data.columns:
        if (x in sample_name or y in sample_name):
            col_list.append(sample_name)
    result = pd.merge(candidate_gene, fpkm_data, how="left", left_index=True, right_on="GeneName").set_index("GeneName")
    col_list = np.concatenate((result.columns[169:], col_list))
    result[col_list].to_csv("result/{}_x_{}_filtered_genes.csv".format(x, y), sep="\t")
